In [1]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tensorflow as tf
from keras.models import model_from_json
from django.http import StreamingHttpResponse
from flask import Flask, Response, render_template
from flask_socketio import SocketIO, emit

In [2]:
signs = {0: "ayubowan", 1: "fifty_2", 2: "fifty_3", 3: "good(hodai)", 4: "his", 5: "house", 6: "i_love_you", 
         7: "naraka", 8: "oba", 9: "ten_3", 10: "thirty_2", 11: "thirty_3", 12: "twenty_2", 13: "twenty_3", 14: "two", 15: "when_1", 16: "when_2"}

sign_model = open('CNN_model.json', 'r')
loaded_json = sign_model.read()
sign_model.close()
sign_language_model = model_from_json(loaded_json)
# weights = np.load('model_weights.npy', allow_pickle=True)
# print('Original data shape:', loaded_json)

sign_language_model.load_weights("CNN_model.h5")

# if weights.shape != (32, 48, 64, 3):
# weights = np.reshape(weights, (1, 10, 1, 1))
# print('Reshaped data shape:', weights.shape)
print("helllo")

helllo


In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

app = Flask(__name__)

@app.route('/')
def index():
    return '<html><body><img src="/video_feed"></body></html>'

def hand_detection():
    cap = cv2.VideoCapture(0)

    with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:

        while cap.isOpened():
            ret, frame = cap.read()

            # changing from BGR to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (1280, 720))

            # flipping the frame
            frame = cv2.flip(frame, 1)
            frame.flags.writeable = False

            # hand detections
            results = hands.process(frame)
            frame.flags.writeable = True

            # changing from RGB to BGR
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for num, hand in enumerate(results.multi_hand_landmarks):
                    mp_drawing.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS,
                                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=3, circle_radius=4),
                                             mp_drawing.DrawingSpec(color=(221, 44, 250), thickness=2, circle_radius=2),
                                             )

                    # Get the coordinates of the topmost hand landmark
                    x, y = int(hand.landmark[0].x), int(hand.landmark[0].y)

                    # Preprocessing the input image
                    frame_processed = cv2.resize(frame, (300, 300))
                    frame_processed = frame_processed.astype("float32") / 255.0
                    frame_processed = np.expand_dims(frame_processed, axis=0)

                    prediction = sign_language_model.predict(frame_processed)
                    predicted_sign = signs[np.argmax(prediction)]
                    maxindex = int(np.argmax(prediction))

                    sign_label = signs[maxindex]
                    cv2.putText(frame, sign_label, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            # Convert the image back to 8-bit unsigned integer type (from float type)
            frame = (frame * 255).astype("uint8")
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
        cap.release()

@app.route('/video_feed')
def video_feed():
    return Response(hand_detection(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
